In [ ]:
# Helpful Encoding :https://towardsdatascience.com/smarter-ways-to-encode-categorical-data-for-machine-learning-part-1-of-3-6dca2f71b159


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin

#!pip install category_encoders
#!pip install xgboost 
import xgboost as xgb

In [2]:
#df = pd.read_csv("../input/train_features.csv", header=0)

df = pd.read_csv("train_features.csv", header=0)
df_test = pd.read_csv("test_features.csv",header=0)
df_labels = pd.read_csv("train_labels.csv", header=0)
df_labels['status_group'].value_counts()
df = df.merge(df_labels, on='id')

In [15]:
df.status_group.value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [11]:
df_slim = df[df.status_group != 'functional needs repair']

In [14]:
df_slim.status_group.value_counts()

functional        32259
non functional    22824
Name: status_group, dtype: int64

## REVISIT THIS DATA. DROP NAN ROWS? 5%

In [17]:
# Based on the most voted answer we can easily define a function that gives us a dataframe to preview the missing values and the % of missing values in each column:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [20]:
missing_values_table(df)

Your selected dataframe has 41 columns.
There are 7 columns that have missing values.


,Missing Values,% of Total Values
scheme_name,28166,47.4
scheme_management,3877,6.5
installer,3655,6.2
funder,3635,6.1
public_meeting,3334,5.6
permit,3056,5.1
subvillage,371,0.6


In [21]:
counts_ = []
features = list(df.columns)

fc = []
for x in features: fc.append([x, df[x].nunique()])
pd.DataFrame(data=fc, columns = ['features','counts']).sort_values(by=['counts']).set_index(['features'])

,counts
features,
recorded_by,1
public_meeting,2
permit,2
status_group,3
source_class,3
quantity_group,5
quantity,5
management_group,5
quality_group,6


In [41]:
# Encode my Y label and return a list of my labels. 
def labeler(dataframe, column):
    from sklearn.preprocessing import LabelEncoder
    dataframe =  dataframe.copy()
    le = LabelEncoder()
    dataframe['labels'] = le.fit_transform(dataframe[column])
    labels = [0,1,2]
    label_names = list(le.inverse_transform(labels))
    label_list = [labels, label_names]
    return dataframe['labels'], label_list


def slimlabeler(dataframe, column):
    from sklearn.preprocessing import LabelEncoder
    dataframe =  dataframe.copy()
    le = LabelEncoder()
    dataframe['labels'] = le.fit_transform(dataframe[column])
    labels = [0,1]
    label_names = list(le.inverse_transform(labels))
    label_list = [labels, label_names]
    return dataframe['labels'], label_list


# Turn Lat/Long into x,y,z, coord plane. 
def lat_long(dataframe):
    from math import cos, sin 
    dataframe =  dataframe.copy()
    dataframe['x_coord'] = dataframe.latitude.apply(lambda x: cos(x)) * dataframe.longitude.apply(lambda x: cos(x))
    dataframe['y_coord'] = dataframe.latitude.apply(lambda x: cos(x)) * dataframe.longitude.apply(lambda x: sin(x))
    dataframe['z_coord'] = dataframe.latitude.apply(lambda x: sin(x))
    dataframe = dataframe.drop(columns=['latitude', 'longitude'])
    return dataframe

# Fix silly boolean issue. 
def no_bool(dataframe, columns):
    dataframe =  dataframe.copy()
    for column in columns:
        dataframe[column] = dataframe[column].replace({True: 'Yes', False: 'No'})
    return dataframe
        

In [23]:
#for i in names:
#    temp_df.loc[temp_df[i].value_counts()[temp_df[i]].values < 20, i] = "RARE_VALUE"

In [25]:
# Now to decide how to handle the following madness:
"""
wpt_name	37400: These have some sites that encompass more than one well. 
date_recorded	356: DROP, missing data. 

# I'll deal with these ones after getting a new baseline. 
lga	125
funder	1897 - Trim?
    print(data.funder.value_counts()[data.funder.value_counts()>777])
    print(len(list(data.funder.value_counts()[data.funder.value_counts()<777])))
ward	2092 
installer	2145
scheme_name	2696 drop
subvillage	19287
""";

In [ ]:
df.waterpoint_type.value_counts()

In [55]:
df.region.value_counts()

Iringa           5294
Shinyanga        4982
Mbeya            4639
Kilimanjaro      4379
Morogoro         4006
Arusha           3350
Kagera           3316
Mwanza           3102
Kigoma           2816
Ruvuma           2640
Pwani            2635
Tanga            2547
Dodoma           2201
Singida          2093
Mara             1969
Tabora           1959
Rukwa            1808
Mtwara           1730
Manyara          1583
Lindi            1546
Dar es Salaam     805
Name: region, dtype: int64

In [ ]:
df.ward.value_counts()

In [ ]:
## Feature	value_count()

# DROP status_group	3 : 
# DROP id	59400: because duh. 
# DROP recorded_by	1 : Only one source. 

# KEEP public_meeting	2 : Boolean to object. 
# KEEP permit	2: Boolean to object.  

# KEEP source_class	3 
# KEEP source	10: These two are sufficiently different. 
# DROP source_type	7 : This one is source but with less detail. Not ordered. 

# DROP quantity_group	5 : These two are exactly the same. Drop either. 
# KEEP  quantity	5 : enough, insufficient, dry, seasonal, unknown  - possibly cat/ordinal

# KEEP management	12 : Most detailed, not ordered. 
# DROP management_group	5 : less-detailed, summary-grouping. Not ordered.
# DROP scheme_management	12 : Missing Values, similar info to management. 

# DROP payment	7
# KEEP payment_type	7 : Shorter Column headers. Identical otherwise.

# KEEP water_quality	8
# DROP quality_group	6

# DROP waterpoint_type_group	6
# KEEP waterpoint_type	7

# DROP extraction_type_class	7
# DROP extraction_type_group	13
# KEEP extraction_type	18


### GEOGRAPHY 
# DROP longitude	57516: Replace with X,Y,Z coordinates
# DROP latitude	57517

# KEEP basin	9 - Non-Regional boarder may overlap with some other indicators of location. 

# DROP region	21 
# KEEP region_code	27: Region Code contains more detail than Region. 

# DROP district_code	20
# KEEP lga	125 : Lga is district with urban/rural flag added in on some values. Lga is most detailed.

# ward	2092
# gps_height	2428
# subvillage	19287
# wpt_name	37400


### OTHER
# construction_year	55
# num_private	65
# amount_tsh	98
# date_recorded	356
# population	1049


### High Cardinality
# funder	1897
# installer	2145
# PDROP  scheme_name	2696 - High cardinality, low 



In [150]:
# Fix or Enhance Features
data = df_slim
data = lat_long(data)
data = no_bool(data, ['permit', 'public_meeting'])

# Define those datasets
X = data
y, label_list = slimlabeler(df_slim, 'status_group')
y.head()

# feature thoughts
nope = ['recorded_by','construction_year', 'payment_type', 'quantity_group','source_type']
good = ['permit']
first = ['extraction_type','management', 'payment','water_quality', 'quantity','source', 'source_class','waterpoint_type']
second = ['extraction_type_group','management_group','quality_group','waterpoint_type_group']
third = ['extraction_type_class']

# Passthrough features. These are any I don't want to mess with. 
passthrough_features = []

#Testing
to_be_tested = ['subvillage']
test_remove = second + third
test_add_oh = []
test_add_binary = []
test_add_numeric = []

# Ones that aren't actually numeric.
binary_features = ['region_code','installer','scheme_name','date_recorded','lga','subvillage']
hashing_features = []

# Dropping features
drop_features_tune = ['id', 'status_group','recorded_by',"region",'funder','district_code',]
drop_features_testing = ['wpt_name']
drop_features = drop_features_tune + test_remove + drop_features_testing

# my need to convert region_code to 

# Ones to binary encode (high cardinality)
#binary_features = ['funder','installer','scheme_name', 'subvillage','region_code', 'district_code']

# Defining my one-hot variables. 
one_hot_features = list(X.select_dtypes(include=['object']))
numeric_features = list(X.select_dtypes(include=['float64', 'int64']))

# Remove features from duplicates if present. 
for x in drop_features:
    if x in one_hot_features: one_hot_features.remove(x)
    if x in numeric_features: numeric_features.remove(x)
    if x in binary_features: binary_features.remove(x)
    if x in hashing_features: hashing_features.remove(x)

for x in binary_features:
    if x in one_hot_features: one_hot_features.remove(x)
    if x in numeric_features: numeric_features.remove(x)
        
for x in hashing_features: 
    if x in one_hot_features: one_hot_features.remove(x)
    if x in numeric_features: numeric_features.remove(x)

binary_features = binary_features + one_hot_features
# List features
the_selected_features = passthrough_features + one_hot_features + numeric_features + binary_features + hashing_features

In [151]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, RobustScaler, StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import f_classif, chi2, SelectKBest
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from category_encoders.hashing import HashingEncoder
from category_encoders.binary import BinaryEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

# Preprocessing pipelines for both numeric and categorical data.
# Using column_transformer https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html


# Define my custom pipeline functions for each type of data. Columns not expressly included are dropped.  


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

polynom_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()), 
    ('polynom', PolynomialFeatures())])

one_hot_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', OrdinalEncoder()),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('binary', BinaryEncoder(drop_invariant=True,return_df=True))])

hashing_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('hashing', HashingEncoder())])

# Create preprocessor pipeline
PreProcessor = ColumnTransformer(
    transformers=[
#        ('pass', 'passthrough', passthrough_features),
        ('drop', 'drop', drop_features),
        #('hash', hashing_transformer, hashing_features),
        ('biy', binary_transformer, binary_features),
        ('num', numeric_transformer, numeric_features)
        #('o-h', one_hot_transformer, one_hot_features)
    ],
    n_jobs = -2)

# Lets test it.

In [152]:
# Test Train Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
print(X_train.shape, y_train.shape)
PreProcessor.fit_transform(X_train, y_train).shape

(49574, 42) (49574,)


(49574, 124)

## Feature Baseline

In [153]:
clf = make_pipeline(PreProcessor, LogisticRegression(max_iter=1000))

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_train, y_train))
print("model score: %.3f" % clf.score(X_val, y_val))

/home/britton/.conda/envs/basicNLP/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


model score: 0.775
model score: 0.782


## After Baseline

In [154]:
clf2 = make_pipeline(PreProcessor, XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.02,  
                      colsample_bytree = 0.3,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=100, 
                      reg_alpha = 0.4,
                      max_depth=17, 
                      gamma=1))

clf2.fit(X_train, y_train)
print("model score: %.3f" % clf2.score(X_train, y_train))
print("model score: %.3f" % clf2.score(X_val, y_val))

[23:50:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2198 extra nodes, 436 pruned nodes, max_depth=17
[23:50:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2950 extra nodes, 588 pruned nodes, max_depth=17
[23:50:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2654 extra nodes, 600 pruned nodes, max_depth=17
[23:50:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2922 extra nodes, 578 pruned nodes, max_depth=17
[23:50:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2710 extra nodes, 618 pruned nodes, max_depth=17
[23:50:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3172 extra nodes, 700 pruned nodes, max_depth=17
[23:50:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2982 extra nodes, 614 pruned nodes, max_depth=17
[23:50:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3392 extra nodes, 796 pru

KeyboardInterrupt: 

[23:50:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2652 extra nodes, 842 pruned nodes, max_depth=17
[23:50:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2624 extra nodes, 680 pruned nodes, max_depth=17
[23:50:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2634 extra nodes, 844 pruned nodes, max_depth=17
[23:50:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3560 extra nodes, 1406 pruned nodes, max_depth=17
[23:50:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2640 extra nodes, 752 pruned nodes, max_depth=17
[23:50:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3110 extra nodes, 958 pruned nodes, max_depth=17
[23:50:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2686 extra nodes, 774 pruned nodes, max_depth=17
[23:50:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3026 extra nodes, 982 pr

[23:50:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2350 extra nodes, 1164 pruned nodes, max_depth=17
[23:50:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2514 extra nodes, 1310 pruned nodes, max_depth=17
[23:50:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2424 extra nodes, 1588 pruned nodes, max_depth=17
[23:50:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2882 extra nodes, 1710 pruned nodes, max_depth=17
[23:50:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2494 extra nodes, 1824 pruned nodes, max_depth=17
[23:50:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2228 extra nodes, 2324 pruned nodes, max_depth=17
[23:50:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2244 extra nodes, 1734 pruned nodes, max_depth=17
[23:50:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2372 extra nodes, 

In [165]:
clf2 = make_pipeline(PreProcessor, XGBClassifier(silent=False,scale_pos_weight=1,learning_rate=0.01, colsample_bytree = 0.3, subsample = 0.8,objective='binary:logistic', n_estimators=450, reg_alpha = 0.1, max_depth=17, gamma=1))

clf2.fit(X_train, y_train)
print("model score: %.3f" % clf2.score(X_train, y_train))
print("model score: %.3f" % clf2.score(X_val, y_val))

[02:04:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2460 extra nodes, 414 pruned nodes, max_depth=17
[02:04:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3242 extra nodes, 580 pruned nodes, max_depth=17
[02:04:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3008 extra nodes, 550 pruned nodes, max_depth=17
[02:04:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3210 extra nodes, 498 pruned nodes, max_depth=17
[02:04:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3102 extra nodes, 522 pruned nodes, max_depth=17
[02:04:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3576 extra nodes, 612 pruned nodes, max_depth=17
[02:04:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3424 extra nodes, 588 pruned nodes, max_depth=17
[02:04:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3796 extra nodes, 622 pru

[02:04:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3764 extra nodes, 1832 pruned nodes, max_depth=17
[02:04:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3072 extra nodes, 820 pruned nodes, max_depth=17
[02:04:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2952 extra nodes, 838 pruned nodes, max_depth=17
[02:04:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3554 extra nodes, 922 pruned nodes, max_depth=17
[02:04:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3676 extra nodes, 1088 pruned nodes, max_depth=17
[02:04:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2986 extra nodes, 1264 pruned nodes, max_depth=17
[02:04:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3172 extra nodes, 882 pruned nodes, max_depth=17
[02:04:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3140 extra nodes, 898 

[02:04:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2862 extra nodes, 1842 pruned nodes, max_depth=17
[02:05:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3172 extra nodes, 1218 pruned nodes, max_depth=17
[02:05:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2584 extra nodes, 1746 pruned nodes, max_depth=17
[02:05:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2736 extra nodes, 1504 pruned nodes, max_depth=17
[02:05:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3286 extra nodes, 1628 pruned nodes, max_depth=17
[02:05:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3472 extra nodes, 1492 pruned nodes, max_depth=17
[02:05:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2944 extra nodes, 1114 pruned nodes, max_depth=17
[02:05:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2794 extra nodes, 

[02:05:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1890 extra nodes, 1782 pruned nodes, max_depth=17
[02:05:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2878 extra nodes, 1852 pruned nodes, max_depth=17
[02:05:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2498 extra nodes, 1276 pruned nodes, max_depth=17
[02:05:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2538 extra nodes, 1174 pruned nodes, max_depth=17
[02:05:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2972 extra nodes, 2188 pruned nodes, max_depth=17
[02:05:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2612 extra nodes, 1666 pruned nodes, max_depth=17
[02:05:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2828 extra nodes, 1418 pruned nodes, max_depth=17
[02:05:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2384 extra nodes, 

[02:05:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2622 extra nodes, 1746 pruned nodes, max_depth=17
[02:05:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2202 extra nodes, 1820 pruned nodes, max_depth=17
[02:05:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2022 extra nodes, 1706 pruned nodes, max_depth=17
[02:05:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2226 extra nodes, 1678 pruned nodes, max_depth=17
[02:05:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1954 extra nodes, 1876 pruned nodes, max_depth=17
[02:05:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2216 extra nodes, 1314 pruned nodes, max_depth=17
[02:05:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1930 extra nodes, 1184 pruned nodes, max_depth=17
[02:05:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2264 extra nodes, 

[02:06:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1762 extra nodes, 1742 pruned nodes, max_depth=17
[02:06:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2032 extra nodes, 1498 pruned nodes, max_depth=17
[02:06:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1806 extra nodes, 2048 pruned nodes, max_depth=17
[02:06:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1700 extra nodes, 1444 pruned nodes, max_depth=17
[02:06:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1800 extra nodes, 1322 pruned nodes, max_depth=17
[02:06:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2096 extra nodes, 1798 pruned nodes, max_depth=17
[02:06:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1964 extra nodes, 1952 pruned nodes, max_depth=17
[02:06:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1990 extra nodes, 

[02:06:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1850 extra nodes, 1718 pruned nodes, max_depth=17
[02:06:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1738 extra nodes, 1584 pruned nodes, max_depth=17
[02:06:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1734 extra nodes, 1684 pruned nodes, max_depth=17
[02:06:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1620 extra nodes, 1338 pruned nodes, max_depth=17
[02:06:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1456 extra nodes, 1186 pruned nodes, max_depth=17
[02:06:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1536 extra nodes, 2174 pruned nodes, max_depth=17
[02:06:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1564 extra nodes, 1576 pruned nodes, max_depth=17
[02:06:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1610 extra nodes, 

[02:06:50] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1566 extra nodes, 2022 pruned nodes, max_depth=17
[02:06:50] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1166 extra nodes, 1888 pruned nodes, max_depth=17
model score: 0.964
model score: 0.869


In [ ]:
## First Tune the Depth

In [160]:
## XGBoost Tuning: Tree Max Depth
XGPipe = make_pipeline(PreProcessor, XGBClassifier(silent=False,scale_pos_weight=1,learning_rate=0.01,  colsample_bytree = 0.8,subsample = 0.8,objective='binary:logistic', n_estimators=100, reg_alpha = 0.4,max_depth=17, gamma=1))

param_grid = {
    'xgbclassifier__max_depth':[5,10,15,17,19,20], # 20, 
    #'xgbclassifier__n_estimators':[50, 150, 250, 350, 450],
    #'xgbclassifier__colsample_bytree':[.3, .8, .9, 1.0],
    #'xgbclassifier__learning_rate':[0.001, 0.0045, 0.0065, 0.010], 
    #'xgbclassifier__reg_lambda':[0, 0.01, 0.10, 0.50, 1]
    }
grid = {}

# Fit on the train set, with grid search cross-validation
XGsearch = GridSearchCV(XGPipe, param_grid=param_grid, cv=3, scoring='accuracy', verbose=10, n_jobs=-2)

XGsearch.fit(X_train, y_train)
print("train model score: %.3f" % XGsearch.score(X_train, y_train))
print("validate model score: %.3f" % XGsearch.score(X_val, y_val))
best_params = XGsearch.best_params_
best_score = XGsearch.best_score_
print(best_params)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of  18 | elapsed:   27.6s remaining:  2.3min
[Parallel(n_jobs=-2)]: Done   5 out of  18 | elapsed:   51.7s remaining:  2.2min
[Parallel(n_jobs=-2)]: Done   7 out of  18 | elapsed:  1.3min remaining:  2.0min
[Parallel(n_jobs=-2)]: Done   9 out of  18 | elapsed:  1.4min remaining:  1.4min
[Parallel(n_jobs=-2)]: Done  11 out of  18 | elapsed:  1.5min remaining:   55.6s
[Parallel(n_jobs=-2)]: Done  13 out of  18 | elapsed:  1.8min remaining:   41.5s
[Parallel(n_jobs=-2)]: Done  15 out of  18 | elapsed:  2.1min remaining:   24.9s
[Parallel(n_jobs=-2)]: Done  18 out of  18 | elapsed:  2.3min finished


[00:55:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3130 extra nodes, 530 pruned nodes, max_depth=20
[00:55:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3330 extra nodes, 580 pruned nodes, max_depth=20
[00:55:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2926 extra nodes, 556 pruned nodes, max_depth=20
[00:55:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3052 extra nodes, 548 pruned nodes, max_depth=20
[00:55:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2992 extra nodes, 468 pruned nodes, max_depth=20
[00:55:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2812 extra nodes, 504 pruned nodes, max_depth=20
[00:55:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2900 extra nodes, 506 pruned nodes, max_depth=20
[00:55:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3046 extra nodes, 496 pru

[00:56:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2956 extra nodes, 834 pruned nodes, max_depth=20
[00:56:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2944 extra nodes, 790 pruned nodes, max_depth=20
[00:56:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2978 extra nodes, 836 pruned nodes, max_depth=20
[00:56:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2968 extra nodes, 816 pruned nodes, max_depth=20
[00:56:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2926 extra nodes, 854 pruned nodes, max_depth=20
[00:56:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3050 extra nodes, 864 pruned nodes, max_depth=20
[00:56:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2952 extra nodes, 972 pruned nodes, max_depth=20
[00:56:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3062 extra nodes, 856 pru

In [166]:
# XGBoost Tuning: Number of Trees(cycles)
XGPipe = make_pipeline(PreProcessor, XGBClassifier(silent=False,scale_pos_weight=1,learning_rate=0.01,  colsample_bytree = 0.8,subsample = 0.8,objective='binary:logistic', n_estimators=100, reg_alpha = 0.4, max_depth=17, gamma=1))

param_grid = {
    'xgbclassifier__n_estimators':[450, 550, 1000]
    #'xgbclassifier__colsample_bytree':[.3, .8, .9, 1.0],
    #'xgbclassifier__learning_rate':[0.001, 0.0045, 0.0065, 0.010], 
    #'xgbclassifier__reg_lambda':[0, 0.01, 0.10, 0.50, 1]
    }
grid = {}

# Fit on the train set, with grid search cross-validation
XGsearch = GridSearchCV(XGPipe, param_grid=param_grid, cv=3, scoring='accuracy', verbose=10, n_jobs=-2)

XGsearch.fit(X_train, y_train)
print("train model score: %.3f" % XGsearch.score(X_train, y_train))
print("validate model score: %.3f" % XGsearch.score(X_val, y_val))
best_params = XGsearch.best_params_
best_score = XGsearch.best_score_
print(best_params)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=-2)]: Done   2 out of   9 | elapsed:  4.8min remaining: 16.9min
[Parallel(n_jobs=-2)]: Done   3 out of   9 | elapsed:  5.6min remaining: 11.2min
[Parallel(n_jobs=-2)]: Done   4 out of   9 | elapsed:  5.6min remaining:  7.0min
[Parallel(n_jobs=-2)]: Done   5 out of   9 | elapsed:  6.0min remaining:  4.8min
[Parallel(n_jobs=-2)]: Done   6 out of   9 | elapsed:  6.4min remaining:  3.2min
[Parallel(n_jobs=-2)]: Done   7 out of   9 | elapsed:  8.8min remaining:  2.5min
[Parallel(n_jobs=-2)]: Done   9 out of   9 | elapsed:  9.0min remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   9 out of   9 | elapsed:  9.0min finished


[02:19:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2778 extra nodes, 414 pruned nodes, max_depth=17
[02:19:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2864 extra nodes, 470 pruned nodes, max_depth=17
[02:19:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2618 extra nodes, 446 pruned nodes, max_depth=17
[02:19:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2828 extra nodes, 470 pruned nodes, max_depth=17
[02:19:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2670 extra nodes, 416 pruned nodes, max_depth=17
[02:19:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2524 extra nodes, 408 pruned nodes, max_depth=17
[02:19:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2692 extra nodes, 406 pruned nodes, max_depth=17
[02:19:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2806 extra nodes, 404 pru

[02:20:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2580 extra nodes, 578 pruned nodes, max_depth=17
[02:20:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2772 extra nodes, 682 pruned nodes, max_depth=17
[02:20:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2616 extra nodes, 640 pruned nodes, max_depth=17
[02:20:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2572 extra nodes, 666 pruned nodes, max_depth=17
[02:20:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2758 extra nodes, 664 pruned nodes, max_depth=17
[02:20:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2880 extra nodes, 776 pruned nodes, max_depth=17
[02:20:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2666 extra nodes, 684 pruned nodes, max_depth=17
[02:20:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2732 extra nodes, 778 pru

[02:21:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2362 extra nodes, 966 pruned nodes, max_depth=17
[02:21:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2260 extra nodes, 1208 pruned nodes, max_depth=17
[02:21:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2514 extra nodes, 972 pruned nodes, max_depth=17
[02:21:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2594 extra nodes, 1060 pruned nodes, max_depth=17
[02:21:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2660 extra nodes, 1106 pruned nodes, max_depth=17
[02:21:12] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2372 extra nodes, 988 pruned nodes, max_depth=17
[02:21:12] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2686 extra nodes, 1098 pruned nodes, max_depth=17
[02:21:13] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2548 extra nodes, 108

...

[02:28:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 424 extra nodes, 778 pruned nodes, max_depth=17
[02:28:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 602 extra nodes, 1254 pruned nodes, max_depth=17
[02:28:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 784 extra nodes, 2036 pruned nodes, max_depth=17
[02:28:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 854 extra nodes, 1836 pruned nodes, max_depth=17
[02:28:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 874 extra nodes, 2050 pruned nodes, max_depth=17
[02:28:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 904 extra nodes, 2118 pruned nodes, max_depth=17
[02:28:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 770 extra nodes, 1598 pruned nodes, max_depth=17
[02:28:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 638 extra nodes, 1220 prun

[02:28:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 660 extra nodes, 1642 pruned nodes, max_depth=17
[02:28:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 336 extra nodes, 772 pruned nodes, max_depth=17
[02:28:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 514 extra nodes, 1386 pruned nodes, max_depth=17
[02:28:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 678 extra nodes, 1424 pruned nodes, max_depth=17
[02:28:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 462 extra nodes, 1518 pruned nodes, max_depth=17
[02:28:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 648 extra nodes, 1750 pruned nodes, max_depth=17
[02:28:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 632 extra nodes, 1476 pruned nodes, max_depth=17
[02:28:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 474 extra nodes, 1296 prun

[02:29:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 426 extra nodes, 1206 pruned nodes, max_depth=17
[02:29:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 454 extra nodes, 1022 pruned nodes, max_depth=17
[02:29:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 636 extra nodes, 2180 pruned nodes, max_depth=17
[02:29:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 626 extra nodes, 1990 pruned nodes, max_depth=17
[02:29:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 462 extra nodes, 1028 pruned nodes, max_depth=17
[02:29:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 452 extra nodes, 1456 pruned nodes, max_depth=17
[02:29:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 624 extra nodes, 1660 pruned nodes, max_depth=17
[02:29:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 704 extra nodes, 2024 pru

[02:30:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 654 extra nodes, 2124 pruned nodes, max_depth=17
[02:30:12] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 464 extra nodes, 1332 pruned nodes, max_depth=17
[02:30:12] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 490 extra nodes, 1644 pruned nodes, max_depth=17
[02:30:13] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 446 extra nodes, 1296 pruned nodes, max_depth=17
[02:30:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 462 extra nodes, 1256 pruned nodes, max_depth=17
[02:30:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 580 extra nodes, 1668 pruned nodes, max_depth=17
[02:30:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 404 extra nodes, 1044 pruned nodes, max_depth=17
[02:30:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 300 extra nodes, 906 prun

[02:30:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 336 extra nodes, 1090 pruned nodes, max_depth=17
[02:30:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 480 extra nodes, 1794 pruned nodes, max_depth=17
[02:30:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 410 extra nodes, 1112 pruned nodes, max_depth=17
[02:30:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 422 extra nodes, 1192 pruned nodes, max_depth=17
[02:30:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 394 extra nodes, 1236 pruned nodes, max_depth=17
[02:30:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 302 extra nodes, 922 pruned nodes, max_depth=17
[02:30:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 550 extra nodes, 1710 pruned nodes, max_depth=17
[02:30:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 592 extra nodes, 2032 prun

In [163]:
## XGBoost Tuning: (colsample)
XGPipe = make_pipeline(PreProcessor, XGBClassifier(silent=False,scale_pos_weight=1,learning_rate=0.01, colsample_bytree = 0.8,subsample = 0.8,objective='binary:logistic', n_estimators=450, reg_alpha = 0.4, max_depth=17, gamma=1))

param_grid = {
    'xgbclassifier__colsample_bytree':[.3, .8, .9, 1.0],
    #'xgbclassifier__learning_rate':[0.001, 0.0045, 0.0065, 0.010], 
    #'xgbclassifier__reg_lambda':[0, 0.01, 0.10, 0.50, 1]
    }
grid = {}

# Fit on the train set, with grid search cross-validation
XGsearch = GridSearchCV(XGPipe, param_grid=param_grid, cv=3, scoring='accuracy', verbose=10, n_jobs=-2)

XGsearch.fit(X_train, y_train)
print("train model score: %.3f" % XGsearch.score(X_train, y_train))
print("validate model score: %.3f" % XGsearch.score(X_val, y_val))
best_params = XGsearch.best_params_
best_score = XGsearch.best_score_
print(best_params)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-2)]: Done   3 out of  12 | elapsed:  2.9min remaining:  8.6min
[Parallel(n_jobs=-2)]: Done   5 out of  12 | elapsed:  5.3min remaining:  7.5min
[Parallel(n_jobs=-2)]: Done   7 out of  12 | elapsed:  5.6min remaining:  4.0min
[Parallel(n_jobs=-2)]: Done   9 out of  12 | elapsed:  5.8min remaining:  1.9min
[Parallel(n_jobs=-2)]: Done  12 out of  12 | elapsed:  7.5min finished


[01:22:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2198 extra nodes, 436 pruned nodes, max_depth=17
[01:22:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2952 extra nodes, 590 pruned nodes, max_depth=17
[01:22:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2670 extra nodes, 596 pruned nodes, max_depth=17
[01:23:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2948 extra nodes, 578 pruned nodes, max_depth=17
[01:23:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2708 extra nodes, 602 pruned nodes, max_depth=17
[01:23:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3062 extra nodes, 650 pruned nodes, max_depth=17
[01:23:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3048 extra nodes, 600 pruned nodes, max_depth=17
[01:23:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3376 extra nodes, 738 pru

[01:23:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3214 extra nodes, 1730 pruned nodes, max_depth=17
[01:23:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2770 extra nodes, 852 pruned nodes, max_depth=17
[01:23:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2550 extra nodes, 784 pruned nodes, max_depth=17
[01:23:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3146 extra nodes, 1024 pruned nodes, max_depth=17
[01:23:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3298 extra nodes, 1102 pruned nodes, max_depth=17
[01:23:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2574 extra nodes, 1254 pruned nodes, max_depth=17
[01:23:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2784 extra nodes, 860 pruned nodes, max_depth=17
[01:23:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2804 extra nodes, 942

[01:23:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2248 extra nodes, 1942 pruned nodes, max_depth=17
[01:23:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2858 extra nodes, 1364 pruned nodes, max_depth=17
[01:23:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2194 extra nodes, 1580 pruned nodes, max_depth=17
[01:23:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2378 extra nodes, 1490 pruned nodes, max_depth=17
[01:23:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2928 extra nodes, 1704 pruned nodes, max_depth=17
[01:23:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3048 extra nodes, 1578 pruned nodes, max_depth=17
[01:23:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2618 extra nodes, 1158 pruned nodes, max_depth=17
[01:23:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2406 extra nodes, 

[01:24:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1510 extra nodes, 1772 pruned nodes, max_depth=17
[01:24:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2522 extra nodes, 1892 pruned nodes, max_depth=17
[01:24:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2184 extra nodes, 1456 pruned nodes, max_depth=17
[01:24:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2246 extra nodes, 1304 pruned nodes, max_depth=17
[01:24:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2652 extra nodes, 2174 pruned nodes, max_depth=17
[01:24:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1994 extra nodes, 1568 pruned nodes, max_depth=17
[01:24:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2496 extra nodes, 1528 pruned nodes, max_depth=17
[01:24:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1988 extra nodes, 

[01:24:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2126 extra nodes, 1920 pruned nodes, max_depth=17
[01:24:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1942 extra nodes, 2028 pruned nodes, max_depth=17
[01:24:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1628 extra nodes, 1780 pruned nodes, max_depth=17
[01:24:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2080 extra nodes, 1854 pruned nodes, max_depth=17
[01:24:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1672 extra nodes, 1968 pruned nodes, max_depth=17
[01:24:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1780 extra nodes, 1398 pruned nodes, max_depth=17
[01:24:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1820 extra nodes, 1294 pruned nodes, max_depth=17
[01:24:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1976 extra nodes, 

[01:24:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1536 extra nodes, 1928 pruned nodes, max_depth=17
[01:24:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1808 extra nodes, 1704 pruned nodes, max_depth=17
[01:24:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1580 extra nodes, 2392 pruned nodes, max_depth=17
[01:24:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1414 extra nodes, 1486 pruned nodes, max_depth=17
[01:24:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1630 extra nodes, 1468 pruned nodes, max_depth=17
[01:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1818 extra nodes, 2380 pruned nodes, max_depth=17
[01:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1534 extra nodes, 2198 pruned nodes, max_depth=17
[01:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1706 extra nodes, 

[01:25:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1690 extra nodes, 1834 pruned nodes, max_depth=17
[01:25:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1514 extra nodes, 1722 pruned nodes, max_depth=17
[01:25:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1676 extra nodes, 1862 pruned nodes, max_depth=17
[01:25:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1412 extra nodes, 1572 pruned nodes, max_depth=17
[01:25:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1426 extra nodes, 1248 pruned nodes, max_depth=17
[01:25:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1226 extra nodes, 2250 pruned nodes, max_depth=17
[01:25:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1428 extra nodes, 1840 pruned nodes, max_depth=17
[01:25:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1370 extra nodes, 

[01:25:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1384 extra nodes, 2384 pruned nodes, max_depth=17
[01:25:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1026 extra nodes, 1962 pruned nodes, max_depth=17
train model score: 0.955
validate model score: 0.869
{'xgbclassifier__colsample_bytree': 0.3}


In [164]:
## XGBoost Tuning: (colsample)
XGPipe = make_pipeline(PreProcessor, XGBClassifier(silent=False,scale_pos_weight=1,learning_rate=0.01, colsample_bytree = 0.3, subsample = 0.8,objective='binary:logistic', n_estimators=450, reg_alpha = 0.1, max_depth=17, gamma=1))

param_grid = {
    'xgbclassifier__learning_rate':[0.001, 0.0045, 0.0065, 0.010], 
    'xgbclassifier__reg_lambda':[0, 0.01, 0.10, 0.50, 1]
    }
grid = {}

# Fit on the train set, with grid search cross-validation
XGsearch = GridSearchCV(XGPipe, param_grid=param_grid, cv=3, scoring='accuracy', verbose=10, n_jobs=-2)

XGsearch.fit(X_train, y_train)
print("train model score: %.3f" % XGsearch.score(X_train, y_train))
print("validate model score: %.3f" % XGsearch.score(X_val, y_val))
best_params = XGsearch.best_params_
best_score = XGsearch.best_score_
print(best_params)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-2)]: Done  10 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-2)]: Done  19 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-2)]: Done  28 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-2)]: Done  39 tasks      | elapsed: 11.5min
[Parallel(n_jobs=-2)]: Done  46 out of  60 | elapsed: 14.2min remaining:  4.3min
[Parallel(n_jobs=-2)]: Done  53 out of  60 | elapsed: 14.5min remaining:  1.9min
[Parallel(n_jobs=-2)]: Done  60 out of  60 | elapsed: 16.2min remaining:    0.0s
[Parallel(n_jobs=-2)]: Done  60 out of  60 | elapsed: 16.2min finished


[01:56:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2888 extra nodes, 380 pruned nodes, max_depth=17
[01:56:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3746 extra nodes, 512 pruned nodes, max_depth=17
[01:56:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3600 extra nodes, 522 pruned nodes, max_depth=17
[01:56:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3750 extra nodes, 380 pruned nodes, max_depth=17
[01:56:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3632 extra nodes, 518 pruned nodes, max_depth=17
[01:56:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4146 extra nodes, 592 pruned nodes, max_depth=17
[01:56:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3896 extra nodes, 438 pruned nodes, max_depth=17
[01:56:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4400 extra nodes, 538 pru

[01:57:13] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4042 extra nodes, 1836 pruned nodes, max_depth=17
[01:57:13] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3614 extra nodes, 786 pruned nodes, max_depth=17
[01:57:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3370 extra nodes, 730 pruned nodes, max_depth=17
[01:57:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3894 extra nodes, 942 pruned nodes, max_depth=17
[01:57:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4164 extra nodes, 996 pruned nodes, max_depth=17
[01:57:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3350 extra nodes, 1336 pruned nodes, max_depth=17
[01:57:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3584 extra nodes, 812 pruned nodes, max_depth=17
[01:57:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3574 extra nodes, 882 p

[01:57:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2990 extra nodes, 1972 pruned nodes, max_depth=17
[01:57:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3290 extra nodes, 1140 pruned nodes, max_depth=17
[01:57:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2640 extra nodes, 1896 pruned nodes, max_depth=17
[01:57:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2890 extra nodes, 1584 pruned nodes, max_depth=17
[01:57:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3484 extra nodes, 1564 pruned nodes, max_depth=17
[01:57:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3670 extra nodes, 1368 pruned nodes, max_depth=17
[01:57:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3026 extra nodes, 1102 pruned nodes, max_depth=17
[01:57:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2916 extra nodes, 

[01:57:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1810 extra nodes, 1876 pruned nodes, max_depth=17
[01:57:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2938 extra nodes, 1740 pruned nodes, max_depth=17
[01:57:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2526 extra nodes, 1336 pruned nodes, max_depth=17
[01:57:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2716 extra nodes, 1190 pruned nodes, max_depth=17
[01:57:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3160 extra nodes, 2102 pruned nodes, max_depth=17
[01:57:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2540 extra nodes, 1676 pruned nodes, max_depth=17
[01:58:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2858 extra nodes, 1332 pruned nodes, max_depth=17
[01:58:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2228 extra nodes, 

[01:58:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2412 extra nodes, 1872 pruned nodes, max_depth=17
[01:58:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2138 extra nodes, 1888 pruned nodes, max_depth=17
[01:58:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1962 extra nodes, 1806 pruned nodes, max_depth=17
[01:58:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2104 extra nodes, 1730 pruned nodes, max_depth=17
[01:58:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1804 extra nodes, 2020 pruned nodes, max_depth=17
[01:58:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2076 extra nodes, 1326 pruned nodes, max_depth=17
[01:58:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1878 extra nodes, 1296 pruned nodes, max_depth=17
[01:58:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2196 extra nodes, 

[01:58:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1772 extra nodes, 1792 pruned nodes, max_depth=17
[01:58:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1964 extra nodes, 1606 pruned nodes, max_depth=17
[01:58:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1760 extra nodes, 2326 pruned nodes, max_depth=17
[01:58:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1576 extra nodes, 1492 pruned nodes, max_depth=17
[01:58:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1722 extra nodes, 1414 pruned nodes, max_depth=17
[01:58:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2044 extra nodes, 1946 pruned nodes, max_depth=17
[01:58:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1772 extra nodes, 2016 pruned nodes, max_depth=17
[01:58:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1814 extra nodes, 

[01:59:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1784 extra nodes, 1688 pruned nodes, max_depth=17
[01:59:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1606 extra nodes, 1656 pruned nodes, max_depth=17
[01:59:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1612 extra nodes, 1614 pruned nodes, max_depth=17
[01:59:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1654 extra nodes, 1528 pruned nodes, max_depth=17
[01:59:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1294 extra nodes, 1114 pruned nodes, max_depth=17
[01:59:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1350 extra nodes, 2192 pruned nodes, max_depth=17
[01:59:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1452 extra nodes, 1632 pruned nodes, max_depth=17
[01:59:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1422 extra nodes, 

[01:59:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1388 extra nodes, 2050 pruned nodes, max_depth=17
[01:59:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1058 extra nodes, 2142 pruned nodes, max_depth=17
train model score: 0.971
validate model score: 0.869
{'xgbclassifier__learning_rate': 0.01, 'xgbclassifier__reg_lambda': 0}


In [ ]:
clf2.named_steps['xgbclassifier'].feature_importances_

In [ ]:
pipe2 = make_pipeline(
    PreProcessor,  
    LogisticRegression())

param_grid2 = {
    'logisticregression__C': [0.01, 0.1, 1, 3 10]
    }

# Fit on the train set, with grid search cross-validation
gs2 = GridSearchCV(pipe2, param_grid=param_grid2, cv=3, 
                      scoring='accuracy', 
                      verbose=10, n_jobs=-2)
gs2.fit(X_train, y_train)


print("model score: %.3f" % gs2.score(X_train, y_train))
print("model score: %.3f" % gs2.score(X_val, y_val))

### Trying with Ridge Classifier

In [ ]:
pipe = make_pipeline(
    PreProcessor, 
    SelectKBest(f_classif), 
    RidgeClassifier())

param_grid = {
    'selectkbest__k': range(1, len(X_train.columns)+1), 
    'ridgeclassifier__alpha': [0.001, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
    }

# Fit on the train set, with grid search cross-validation
gs = GridSearchCV(pipe, param_grid=param_grid, cv=5, 
                      scoring='accuracy', 
                      verbose=10, n_jobs=-2)
gs.fit(X_train, y_train)
print("model score: %.3f" % gs.score(X_train, y_train))
print("model score: %.3f" % gs.score(X_val, y_val))

### Deep diving on some of these features.


In [ ]:
print(data.ward.value_counts()[data.funder.value_counts()>777])
print(len(list(data.ward.value_counts()[data.funder.value_counts()<777])))

In [ ]:
print(data.wpt_name.value_counts()[data.wpt_name.value_counts()>100])
print(len(list(data.wpt_name.value_counts()[data.wpt_name.value_counts()<100])))

In [ ]:
data.T.head(40)

In [ ]:
data.waterpoint_type.value_counts()

In [ ]:
df.region_code.nunique()

In [ ]:
from sklearn.feature_selection import RFECV
clf8 = make_pipeline(PreProcessor, RFECV(estimator=LogisticRegression(), step=1, min_features_to_select=1, cv=3, scoring='accuracy', verbose=10, n_jobs=-2))
clf8.fit(X_train, y_train)
print("model score: %.3f" % clf8.score(X_train, y_train))
print("model score: %.3f" % clf8.score(X_val, y_val))

## Pre-process the test data, make predictions, and format submission

In [171]:
# XGBoost Tuning: Number of Trees(cycles)
XGPipe = make_pipeline(PreProcessor, XGBClassifier(silent=False,scale_pos_weight=1,learning_rate=0.01,  colsample_bytree = 0.3, subsample = 0.8, objective='binary:logistic', n_estimators=5000, reg_alpha = 0.4, max_depth=30, gamma=1))
XGPipe.fit(X, y)

[02:46:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4648 extra nodes, 1402 pruned nodes, max_depth=23
[02:46:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3118 extra nodes, 792 pruned nodes, max_depth=23
[02:46:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3536 extra nodes, 978 pruned nodes, max_depth=22
[02:46:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3322 extra nodes, 754 pruned nodes, max_depth=23
[02:46:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4008 extra nodes, 858 pruned nodes, max_depth=25
[02:46:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3496 extra nodes, 864 pruned nodes, max_depth=28
[02:46:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3318 extra nodes, 1002 pruned nodes, max_depth=23
[02:46:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4104 extra nodes, 938 p

[02:46:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3074 extra nodes, 1260 pruned nodes, max_depth=24
[02:46:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3772 extra nodes, 1296 pruned nodes, max_depth=24
[02:46:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3810 extra nodes, 1548 pruned nodes, max_depth=28
[02:46:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3818 extra nodes, 1496 pruned nodes, max_depth=29
[02:46:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3462 extra nodes, 1396 pruned nodes, max_depth=23
[02:46:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2934 extra nodes, 1800 pruned nodes, max_depth=20
[02:46:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2828 extra nodes, 1688 pruned nodes, max_depth=23
[02:46:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3552 extra nodes, 

[02:47:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2304 extra nodes, 2152 pruned nodes, max_depth=20
[02:47:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2716 extra nodes, 2418 pruned nodes, max_depth=21
[02:47:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3548 extra nodes, 2228 pruned nodes, max_depth=22
[02:47:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3308 extra nodes, 1584 pruned nodes, max_depth=30
[02:47:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3416 extra nodes, 1828 pruned nodes, max_depth=25
[02:47:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3050 extra nodes, 2156 pruned nodes, max_depth=23
[02:47:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2950 extra nodes, 1952 pruned nodes, max_depth=23
[02:47:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3532 extra nodes, 

[02:47:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2740 extra nodes, 2376 pruned nodes, max_depth=23
[02:47:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2854 extra nodes, 2498 pruned nodes, max_depth=21
[02:47:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2846 extra nodes, 3152 pruned nodes, max_depth=20
[02:47:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2434 extra nodes, 3166 pruned nodes, max_depth=22
[02:47:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2636 extra nodes, 2186 pruned nodes, max_depth=23
[02:47:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2008 extra nodes, 2708 pruned nodes, max_depth=20
[02:47:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2938 extra nodes, 2248 pruned nodes, max_depth=26
[02:47:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2694 extra nodes, 

[02:48:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1962 extra nodes, 3344 pruned nodes, max_depth=20
[02:48:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2498 extra nodes, 2524 pruned nodes, max_depth=23
[02:48:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2330 extra nodes, 3092 pruned nodes, max_depth=23
[02:48:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2522 extra nodes, 2402 pruned nodes, max_depth=25
[02:48:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2444 extra nodes, 2430 pruned nodes, max_depth=23
[02:48:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2464 extra nodes, 2426 pruned nodes, max_depth=25
[02:48:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2502 extra nodes, 2736 pruned nodes, max_depth=29
[02:48:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1818 extra nodes, 

...

[03:23:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 2106 pruned nodes, max_depth=10
[03:23:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 2264 pruned nodes, max_depth=30
[03:23:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 2362 pruned nodes, max_depth=22
[03:23:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 2142 pruned nodes, max_depth=30
[03:23:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 2334 pruned nodes, max_depth=22
[03:23:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 1460 pruned nodes, max_depth=22
[03:23:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 2002 pruned nodes, max_depth=8
[03:23:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 1904 pruned 

[03:24:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 2400 pruned nodes, max_depth=21
[03:24:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 2014 pruned nodes, max_depth=15
[03:24:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 2086 pruned nodes, max_depth=16
[03:24:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 2218 pruned nodes, max_depth=16
[03:24:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 2330 pruned nodes, max_depth=11
[03:24:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 2386 pruned nodes, max_depth=16
[03:24:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 146 extra nodes, 2174 pruned nodes, max_depth=24
[03:24:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 2286 pruned n

[03:24:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 2246 pruned nodes, max_depth=25
[03:24:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 2352 pruned nodes, max_depth=12
[03:24:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 2146 pruned nodes, max_depth=19
[03:24:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 2296 pruned nodes, max_depth=16
[03:24:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 2310 pruned nodes, max_depth=19
[03:24:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 2346 pruned nodes, max_depth=20
[03:24:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 2336 pruned nodes, max_depth=25
[03:24:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 2064 pruned no

[03:25:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 1960 pruned nodes, max_depth=14
[03:25:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 1914 pruned nodes, max_depth=9
[03:25:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 2354 pruned nodes, max_depth=15
[03:25:12] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 2340 pruned nodes, max_depth=19
[03:25:12] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 2334 pruned nodes, max_depth=16
[03:25:13] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 2162 pruned nodes, max_depth=12
[03:25:13] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 2436 pruned nodes, max_depth=12
[03:25:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 2312 pruned node

Pipeline(memory=None,
     steps=[('columntransformer', ColumnTransformer(n_jobs=-2, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('drop', 'drop', ['id', 'status_group', 'recorded_by', 'region', 'funder', 'district_code', 'extraction_type_group', 'management_group', 'qualit...   reg_alpha=0.4, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=0.8))])

In [172]:
print(df_test.shape)
test_data = lat_long(df_test)
test_data['status_group'] = df_test.id
test_data = no_bool(test_data, ['permit', 'public_meeting'])
#test_data = PreProcessor.fit_transform(test_data)
print(test_data.shape)

(14358, 40)
(14358, 42)


In [173]:
#Make predictions using the features from the test data set
predictions = XGPipe.predict(test_data)

#Display our predictions - they are either 0 or 1 for each training instance 
#depending on whether our algorithm believes the person survived or not.
predictions

# Map our labels back onto the predictions
keys = label_list[0]
values = label_list[1]
status_labels = dict(zip(keys, values))
predictions_mapped = np.vectorize(status_labels.get)(predictions)

In [174]:
#Create a  DataFrame with the passengers ids and our prediction regarding whether they survived or not
submission = pd.DataFrame({'id':df_test['id'],'status_group':predictions_mapped})

# make sure the shapes line up. 
print (df_test.shape)
print (submission.shape)

#Visualize the first 5 rows
submission.head()

(14358, 40)
(14358, 2)


,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [175]:
#Convert DataFrame to a csv file that can be uploaded
#This is saved in the same directory as your notebook
filename = 'submission.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

Saved file: submission.csv


# MISC Ref Material

In [ ]:
def get_column_names_from_ColumnTransformer(column_transformer):    
    col_name = []
    for transformer_in_columns in column_transformer.transformers_[:-1]:#the last transformer is ColumnTransformer's 'remainder'
        raw_col_name = transformer_in_columns[2]
        if isinstance(transformer_in_columns[1],Pipeline): 
            transformer = transformer_in_columns[1].steps[-1][1]
        else:
            transformer = transformer_in_columns[1]
        try:
            names = transformer.get_feature_names()
        except AttributeError: # if no 'get_feature_names' function, use raw column name
            names = raw_col_name
        if isinstance(names,np.ndarray): # eg.
            col_name += names.tolist()
        elif isinstance(names,list):
            col_name += names    
        elif isinstance(names,str):
            col_name.append(names)
    return col_name

get_column_names_from_ColumnTransformer(PreProcessor)

In [ ]:
# Rewriting my Dummy Regression Baseline one as a function
def baseline(data):
    name = "Dummy Regression Baseline"
    # Split data into train and test
    X_train, X_test, y_train, y_test = split(data)

    # Define an estimator and param_grid
    # WHEN DEFINING YOU CAN GIVE IT A NAME OTHERWISE IT WILL USE THE PIPELINE NAME AUTOGEN NAME (name of the function but lowercase)
    pipe = make_pipeline(
        PreProcesser(), 
        DummyRegressor(strategy='mean'))
    pipe.fit(X_train, y_train)

    scorer = 'MAE'

    ### Get the scores with the MAE Function
    y_pred_train = pipe.predict(X_train)  
    y_pred_test = pipe.predict(X_test)

    train_score = mean_absolute_error(y_train, y_pred_train)
    test_score = mean_absolute_error(y_test, y_pred_test)
    score_variance = test_score - train_score
    cv_score = 0.0000000000000
    best_params = pipe.get_params
    best_estimator = ""
    selected_names = list(X_train.columns)
    unselected_names = []

    return [name, scorer, train_score, test_score, score_variance, cv_score, selected_names, unselected_names, best_params, best_estimator]


# Rewriting my GridSearch CV as a function 
def compare(data, name):
    X_train, X_test, y_train, y_test = split(data)

    pipe = make_pipeline(
        PreProcessor, 
        SelectKBest(f_regression), 
        Ridge())

    param_grid = {
        'selectkbest__k': range(1, len(X_train.columns)+1), 
        'ridge__alpha': [0.1, 1.0, 10.]
    }

    scorer = 'MAE'

    # Fit on the train set, with grid search cross-validation
    gs = GridSearchCV(pipe, param_grid=param_grid, cv=3, 
                      scoring='neg_mean_absolute_error', 
                      verbose=0)
    gs.fit(X_train, y_train)

    train_score = -gs.score(X_train, y_train)
    test_score = -gs.score(X_test, y_test)
    score_variance = test_score - train_score
    cv_score = -gs.best_score_
    best_params = gs.best_params_
    best_estimator = gs.best_estimator_

    # selected features? 
    # 'selectkbest' is the autogenerated name of the SelectKBest() function in the pipeline
    selector = gs.best_estimator_.named_steps['selectkbest']
    all_names = X_train.columns

    # get_support returns a mask of the columns in True / False
    selected_mask = selector.get_support()
    # Passing the boolean list as the column names creates a masked list.  
    selected_names = list(all_names[selected_mask])
    unselected_names = list(all_names[~selected_mask])

    return [name, scorer, train_score, test_score, score_variance, cv_score, selected_names, unselected_names, best_params, best_estimator]

In [ ]:
#pd.DataFrame({'Variable':X.columns,
#              'Importance':clf2.named_steps['xgbclassifier'].feature_importances_}).sort_values('Importance', ascending=False)
#pipe.steps[0][1].get_feature_names()